In [1]:
pip install pyttsx3


  Obtaining dependency information for pyttsx3 from https://files.pythonhosted.org/packages/33/9a/de4781245f5ad966646fd276259ef7cfd400ba3cf7d5db7c0d5aab310c20/pyttsx3-2.90-py3-none-any.whl.metadata
  Obtaining dependency information for comtypes from https://files.pythonhosted.org/packages/2b/82/b897453a633dfd25bc09492e91b27f2f2fed5c9ef8d6d5fd7ecdb5148786/comtypes-1.3.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/197.8 kB ? eta -:--:--
   -------- ------------------------------- 41.0/197.8 kB 2.0 MB/s eta 0:00:01
   ---------------------------- ----------- 143.4/197.8 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 197.8/197.8 kB 1.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import mediapipe as mp
import math
import pyttsx3
import time

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

engine = pyttsx3.init()
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[1].id)  # Assuming the female voice is at index 1

def calculateAngle(landmark1, landmark2, landmark3):
    x1, y1 = landmark1.x, landmark1.y
    x2, y2 = landmark2.x, landmark2.y
    x3, y3 = landmark3.x, landmark3.y
    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))
    if angle < 0:
        angle += 360
    return angle

def classifyPose(landmarks):
    left_shoulder = landmarks.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
    left_wrist = landmarks.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST]

    right_shoulder = landmarks.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]
    right_wrist = landmarks.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST]

    if (left_shoulder and left_wrist and right_shoulder and right_wrist):

        # Calculate the slope of the line passing through left shoulder and left wrist
        left_slope = (left_wrist.y - left_shoulder.y) / (left_wrist.x - left_shoulder.x + 1e-10)

        # Calculate the slope of the line passing through right shoulder and right wrist
        right_slope = (right_wrist.y - right_shoulder.y) / (right_wrist.x - right_shoulder.x + 1e-10)

        # Check if the slopes are approximately equal (within a tolerance)
        if abs(left_slope - right_slope) < 0.2:
            return 'T Pose'

    return 'Unknown Pose'

pose_video = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, model_complexity=1)

video = cv2.VideoCapture(0)
cv2.namedWindow('Pose Detection', cv2.WINDOW_NORMAL)

last_command_time = 0

while video.isOpened():
    ok, frame = video.read()
    if not ok:
        break

    frame = cv2.flip(frame, 1)
    frame_height, frame_width, _ = frame.shape
    frame = cv2.resize(frame, (int(frame_width * (640 / frame_height)), 640))

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose_video.process(rgb_frame)

    if results.pose_landmarks:
        pose_label = classifyPose(results)
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        cv2.putText(frame, pose_label, (10, 30), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)
        
        current_time = time.time()
        if current_time - last_command_time >= 5:
            if pose_label == 'T Pose':
                engine.say("T Pose")
            elif pose_label == 'Unknown Pose':
                engine.say("Unknown")
            engine.runAndWait()
            last_command_time = current_time

    cv2.imshow('Pose Detection', frame)

    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break

video.release()
cv2.destroyAllWindows()
